# Assignment 3: Nelson Herrera

In [1]:
import pygame
import moderngl
import numpy as np
import time

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()
# Set the version of OpenGL. Initialize OpenGL context using Pygame
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 4)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 1)
pygame.display.set_mode((800,600), pygame.OPENGL | pygame.DOUBLEBUF)

# Get OpenGL context associated with the window
gl = moderngl.get_context()



2024-09-10 00:01:20.405 Python[5427:101189] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


### Create OpenGL Rendering Contex

### Setting up the rendering pipeline with OpenGL

In [3]:
# Generate t values between -1 and 1 for the full heart shape
t_values = np.linspace(-1, 1, 10000)
t_values = t_values[t_values != 0]  # Exclude zero to avoid log(0)

# Logarithmic heart parametric equations
x_values = np.sin(t_values) * np.cos(t_values) * np.log(np.abs(t_values))

# Ensure we handle negative cos(t) values by taking the square root of non-negative values
cos_t_values = np.cos(t_values)
cos_t_values[cos_t_values < 0] = 0  # Replace negative cos(t) with 0 to avoid sqrt of negative numbers

# Calculate y values
y_values = np.abs(t_values)**0.3 * np.sqrt(cos_t_values) - 0.5

# Scale down x_values to make the heart narrower
x_values = x_values * 0.8  # Adjust this scaling factor as necessary to control width

# Add the center point (0, 0) for the triangle fan
center_point = np.array([0.0, 0.0], dtype='float32')

# The boundary points are the calculated x, y pairs
boundary_points = np.column_stack([x_values, y_values]).astype('float32')

# Combine the center with the boundary points
all_points = np.vstack([center_point, boundary_points])

# Flatten the positions array for OpenGL input
positions = all_points.flatten()
# Upload Vertex data to GPU
vertexBuffer = gl.buffer(positions)

### Create a Shader Programs
- Shaders are pieces of code taht run on your GPU. (written in GLSL)
- We will be making two. A Vertex Shader and a Fragment Shader
#### Vertex Shader:
- Allows us to perform basic processing on the vertex attributes
    - like transformations, scaling, etc.
- Takes a single vertex as input
#### Fragment Shader:
- Allows us to copmute color (RGBA for each fragment)
- Also known as *Pixel Shader*

In [4]:
# Vertex Shader
vertex_shader_code = '''
#version 330 core

layout (location = 0) in vec2 position;
uniform float scale;
void main() {
    gl_Position = vec4(position * scale, 0.0, 1.0);
}
'''

In [5]:
# Fragment Shader
fragment_shader_code = '''
# version 330 core 

out vec4 outColor;

void main() {
    outColor = vec4(1.0, 0.0, 1.0, 1.0);
}
'''

In [6]:
# Store them in the main shader program
shaderProgram = gl.program(
    vertex_shader = vertex_shader_code,
    fragment_shader = fragment_shader_code
)


### Configure Vertex Attributes
- This is the process of connecting the buffer to shader program
    - Specify how vertex attributes are stroed in the vertex buffer:
        - Vertex buffer is just a raw array, We need to tell the shader program how to interpret that data
- We know how our position coordinates are stored in this vertex buffer, but OpenGL doesn't know that

In [7]:
renderable = gl.vertex_array(shaderProgram, [(vertexBuffer, '2f', 'position')])



In [8]:
pygame.display.set_caption("Assignment 3: Nelson Herrera")
clock = pygame.time.Clock()  # Create a clock object

running = True

# Set up zoom variables
zoomFactor = 1.0
deltaZoom = 0.025
zoomingOut = True  # We will toggle this to control zooming in and out

while running:
    clock.tick(60)  # limits FPS to 30
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == 27):
            running = False
            
     # Update zoom factor
    if zoomingOut:
        zoomFactor -= deltaZoom
        if zoomFactor <= 1 / 5:  # Start zooming in when zooming out reaches limit
            zoomingOut = False
    else:
        zoomFactor += deltaZoom
        if zoomFactor >= 5:  # Start zooming out when zooming in reaches limit
            zoomingOut = True

    # Set uniform value for zoom factor
    shaderProgram['scale'] = zoomFactor

    # Clear screen
    gl.clear(0.2, 0.2, 0.2)

    # Render the heart shape
    renderable.render(moderngl.TRIANGLE_FAN)

    # Swap buffers
    pygame.display.flip()

pygame.quit()  # quit after the loop ends
quit() # Needed this to quit the process on my Mac without jupyter notebook freezing